In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, OKXRestAPI, GateIORestAPI
from jaref_bot.strategies.arbitrage import find_tokens_to_open_order, get_open_volume, get_best_prices
from jaref_bot.utils.coins import get_step_info, get_min_qty, round_volume, get_price_scale
from jaref_bot.trading.functions import handle_position, place_limit_order, cancel_order, set_leverage, place_market_order
from jaref_bot.data.data_functions import is_data_up_to_date
from jaref_bot.utils.data import round_size

from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.db.redis_manager import RedisManager
from jaref_bot.config.credentials import host, user, password, db_name

import redis
from redis.exceptions import ConnectionError

import sys
import pandas as pd
import polars as pl
import polars.selectors as cs
import numpy as np
import logging
from datetime import datetime
from time import sleep, time
from decimal import Decimal, ROUND_DOWN
import signal
import random, string
import pickle

from asyncio.exceptions import CancelledError
import aiohttp, asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s")
logging.getLogger('aiohttp').setLevel('ERROR')
logging.getLogger('asyncio').setLevel('ERROR')
logger = logging.getLogger()

In [ ]:
def signal_handler(sig, frame):
    global exit_flag
    print('Завершение работы.')
    exit_flag = True

In [ ]:
def check_open_conditions(token: str, 
                          current_orders: pd.DataFrame, 
                          pending_orders: dict, 
                          min_order: float,
                          max_position: float) -> Decimal:
    """
        Функция проверяет условия для открытия нового ордера. Новый ордер будет открыт если:
        - Токена ещё нет ни в текущих ордерах, ни в открытых позициях
        - Токен есть в открытых позициях, но размер позиции позволяет открыть ещё одну сделку.

        Args:
            token - название токена. Например, 'ADA_USDT'.
            current_orders - датафрейм с текущими позициями.
            pending_orders - словарь с текущими ордерами.
            max_position - максимально возможный размер позиции на бирже.
        Return:
            Размер ордера в usdt, который можно открыть.
    
    """
    # Сначала проверяем условие, что такой ордер ещё не открывали
    token_in_positions = token in current_orders['token'].to_list()
    
    token_in_pending = False
    position_size = Decimal('0')
    for exc, data in pending_orders.items():
        for tok in data:
            if tok == token:
                token_in_pending = True
                position_size += Decimal(data[tok]['qty'])
    if not (token_in_positions or token_in_pending):
        return max_position

    # Проверяем условие, что ордер уже есть среди открытых позиций, но размер позиции позволяет добавить
    price = Decimal(current_orders[current_orders['token'] == token]['price'].max())
    opened_position = Decimal(current_orders[current_orders['token'] == token]['usdt_amount'].max())
    
    position_usdt = float(position_size * price + opened_position)
    if position_usdt < max_usdt_order - min_usdt_order:
        return max_usdt_order - position_usdt
                
    return False

In [ ]:
def check_counter_conditions(token: str, 
                pending_orders: dict, 
                long_exc: str, 
                short_exc: str,
                long_price: float,
                short_price: float) -> bool:
    long_order_status = None
    short_order_status = None
    
    for exc, data in pending_orders.items():
        open_side = data.get(token, {}).get('side', None)
        status = data.get(token, {}).get('status', None)
        price_tick = get_price_scale(coin_information, token, exc + '_linear')
    
        if open_side == 'buy':
            if status == 'filled':
                long_order_status = 'filled'
                order_side = 'sell'
                price = Decimal(short_price) - n_ticks * price_tick
            elif status == 'placed':
                long_order_status = 'placed'
        elif open_side == 'sell':
            if status == 'filled':
                short_order_status = 'filled'
                order_side = 'buy'
                price = Decimal(long_price) + n_ticks * price_tick
            elif status == 'placed':
                short_order_status = 'placed'
    
    # Если ордер открыт только в одну сторону, тогда возвращаем размер и направление встречного ордера
    if (long_order_status == 'filled' and short_order_status is None
               ) or (
        long_order_status is None and short_order_status == 'filled'):

        qty = pending_orders[exc][token]['qty']

        return exc, order_side, qty
    
    return (False, False)

In [ ]:
market_fees = {'bybit_spot': 0.001, 'bybit_linear': 0.001, 'okx_spot': 0.001, 'okx_linear': 0.0005,
               'gate_spot': 0.002, 'gate_linear': 0.0005}

In [ ]:
logger.info('Инициализируем биржи...')
# ====================================
# Инициация нужных криптобирж, рынков и БД
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
exc_manager.add_market("okx_linear", OKXRestAPI('linear'))
exc_manager.add_market("gate_linear", GateIORestAPI('linear'))

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
postgre_manager = DBManager(db_params)

redis_client = redis.Redis(db=0, decode_responses=True)
redis_price_manager = RedisManager(db_name = 'orderbooks')
redis_order_manager = RedisManager(db_name = 'orders')

try: 
    redis_client.ping()
    print('Сервер Redis запущен')
except ConnectionError:
    print('Сервер Redis не отвечает')

In [ ]:
# Подготовительный этап. Загружаем техническую инфу по токенам (мин. кол-во и число знаков после запятой)
coin_information = exc_manager.get_instrument_data()

with open("./data/coin_information.pkl", "wb") as f:
    pickle.dump(coin_information, f)

In [ ]:
# with open("./data/open_order_history.pkl", "rb") as f:
#     orders_history = pickle.load(f)
    
leverage_change_history = []

#### Pair trading

In [ ]:
from jaref_bot.analysis.utils import make_spread_df_bulk, make_trunc_df, make_df_from_orderbooks, make_spread_df
from zoneinfo import ZoneInfo
from datetime import timedelta

In [ ]:
def make_df(df, token_1, token_2):
    df = df.with_columns(
            ((pl.col(f'{token_1}_bid_price') + pl.col(f'{token_1}_ask_price')) / 2).alias(token_1),
            ((pl.col(f'{token_2}_bid_price') + pl.col(f'{token_2}_ask_price')) / 2).alias(token_2),
        ).select('bucket', 'ts', token_1, token_2, 'spread'
        ).rename({'bucket': 'time'})
    return df

In [ ]:
token_1 = 'ARKM'
token_2 = 'SAND'

In [ ]:
while True:
    end_time = datetime.now().replace(tzinfo=ZoneInfo("Europe/Moscow"))
    start_time = end_time - timedelta(minutes = 5 * 120)
    
    # Загрузка исторических данных
    df_1 = postgre_manager.get_orderbooks(exchange='bybit',
                                         market_type='linear',
                                         symbol=token_1 + '_USDT',
                                         interval='5min',
                                         start_date=start_time)
    df_2 = postgre_manager.get_orderbooks(exchange='bybit',
                                         market_type='linear',
                                         symbol=token_2 + '_USDT',
                                         interval='5min',
                                         start_date=start_time)
    df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time, end_time=end_time)
    df = make_df(df, token_1, token_2)

    # Текущий ордербук
    current_data = redis_price_manager.get_orderbooks(1)
    t1 = current_data.filter(pl.col('symbol') == f'{token_1}_USDT')
    t2 = current_data.filter(pl.col('symbol') == f'{token_2}_USDT')# .select('bidprice_0', 'askprice_0').mean_horizontal().item()
    
    t1_time = t1['update_time'].item()
    t2_time = t2['update_time'].item()
    t1_ts = t1['ts'].item()
    t2_ts = t2['ts'].item()
    
    if abs(t1_ts - t2_ts) > 5:
        print('Данные устарели!')
    
    t1_avg_price = t1.select('bidprice_0', 'askprice_0').mean_horizontal().item()
    t2_avg_price = t2.select('bidprice_0', 'askprice_0').mean_horizontal().item()
    spread = np.log(t1_avg_price) - np.log(t2_avg_price)

    row = pl.DataFrame({'time': min(t1_time, t2_time), 'ts': min(t1_ts, t2_ts), 
          token_1: t1_avg_price, token_2: t2_avg_price, 'spread': spread}
              ).with_columns(pl.col('time').dt.replace_time_zone(time_zone="Europe/Moscow"))
    spread_df = df.vstack(row)
    z_score = make_spread_df(spread_df, token_1, token_2, 60).tail(1).select('z_score').item()
    
    ct = datetime.now().strftime('%H:%M:%S')
    print(f'{ct}, z_score: {z_score:.2f}  ', end='\r')

    if z_score > 2.0 or z_score < -2.0:
        print(z_score)
        print(t1)
        print(t2)
        break
    sleep(0.5)

In [ ]:
z_score

In [ ]:
480 * 5 / 60

In [1]:
from random import choice

In [2]:
search_space = (
            ('4h', 8), ('4h', 10), ('4h', 12), ('4h', 16), ('4h', 20),
            ('1h', 12), ('1h', 18), ('1h', 24), ('1h', 36), ('1h', 48), ('1h', 60),
            ('15m', 15), ('15m', 30), ('15m', 45),
            ('15m', 60), ('15m', 80), ('15m', 100), ('15m', 120), ('15m', 160),
            ('5m', 60), ('5m', 90), ('5m', 120), ('5m', 240), ('5m', 360)
        )

In [11]:
tf, wind = choice(search_space)
tf, wind

('15m', 80)

In [3]:
low_in_params = (-0.8, -1.0, -1.2, -1.4, -1.6, -1.8, -2.0, -2.2, -2.4, -2.6, -2.8, -3.0)


In [10]:
choice(low_in_params)

-1.4

#### Open arbitrage orders

In [ ]:
logger.info('Запускаем основной цикл программы...')

arbitrage_tokens = {# 'ADA_USDT': {'long_exc': 'bybit', 'short_exc': 'okx', 'min_edge': 0.06},
                    # 'ALGO_USDT': {'long_exc': 'gate', 'short_exc': 'bybit', 'min_edge': -0.017}
                    'ADA_USDT': {'long_exc': 'gate', 'short_exc': 'bybit', 'min_edge': -10.017}
                   }

exit_flag = False
signal.signal(signal.SIGINT, signal_handler)
# ====================================
# Параметры запуска
limit_type = 'limit' # Первый ордер ставим по лучшей цене, а второй отправляем сразу же, как сматчился первый
n_ticks = Decimal('1')

demo = True
leverage = 2
max_usdt_order = 100
min_usdt_order = 10

max_orders = 3
# ====================================
# Основной цикл
while not exit_flag:
    token, data = None, None
    open_exc, cntr_side, cntr_qty = None, None, None
    long_exc, short_exc, min_edge = None, None, 512

    current_orders = postgre_manager.get_table('current_orders')
    n_current_orders = len(current_orders) // 2
    pending_orders = redis_order_manager.get_pending_orders()
    
    try:
        # Скачиваем текущие данные по ордербукам из Redis
        try:
            current_data = redis_price_manager.get_orderbooks(n_levels=5).filter(pl.col('symbol').is_in(arbitrage_tokens))
        except pl.exceptions.ColumnNotFoundError:
            sleep(1)
            continue

        # Проверка на то, что данные являются актуальными, что ни одна из бирж не подвисла
        if not is_data_up_to_date(current_data, time_thresh=10):
            logger.error(f'Устаревшие данные по ценам.')
            sleep(1)
        if len(current_data) == 0:
            raise Exception('No data!')
        
        for token, data in arbitrage_tokens.items():
            # Настройки направления сделок:
            long_exc = data['long_exc']
            short_exc = data['short_exc']
            min_edge = data['min_edge']

            # Определяем текущие цены
            df_long = current_data.filter((pl.col("exchange") == long_exc) & (pl.col("symbol") == token))
            df_short = current_data.filter((pl.col("exchange") == short_exc) & (pl.col("symbol") == token))
            long_price = df_long.select('askprice_0').item()
            short_price = df_short.select('bidprice_0').item()
            diff = (short_price / long_price - 1) * 100

            # Обработчик открытых в одном направлении лимитных ордеров.
            # ---------------------------------------------------------
            try:
                open_exc, cntr_side, cntr_qty = check_counter_conditions(token, pending_orders, long_exc, short_exc, long_price, short_price)
                cntr_qty = Decimal(cntr_qty)
                
                if open_exc == long_exc:
                    cntr_exc = short_exc
                elif open_exc == short_exc:
                    cntr_exc = long_exc
                
                cntr_ts = int(datetime.timestamp(datetime.now()))
                exc_name = cntr_exc + '_linear'

                resp = place_market_order(demo, exc_name, token, cntr_side, cntr_qty, coin_information)
                logger.info(f'[PLACE OPEN ORDER] {cntr_qty} {token}; {cntr_side} on {cntr_exc}')

                if cntr_exc == 'gate':
                    ord_id = resp['id']
                else:
                    ord_id = resp

                redis_order_manager.add_order(cntr_exc, token, float(cntr_qty), cntr_side, ord_id, cntr_ts, status='placed')
            
            except ValueError:
                pass                
            
            # Обработчик новых ордеров
            # -----------------------------------------------------------------------------            
            usdt_amount = check_open_conditions(token, current_orders, pending_orders, min_usdt_order, max_usdt_order)
            if usdt_amount:                
                long_exc_full = long_exc + '_linear'
                short_exc_full = short_exc + '_linear'
                
                ct = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                min_qty_step = get_step_info(coin_information, token, long_exc_full, short_exc_full)
                
                vol = leverage * float(usdt_amount) / max(long_price, short_price)
                volume = round_volume(volume=vol, qty_step=min_qty_step) 
                                
                usdt_amount = volume / Decimal(volume) * Decimal(max_usdt_order)

                # Сначала на обеих биржах меняем плечо, чтобы если есть проблемы со связью,
                #   всё отвалилось на этапе изменения плеча, а не после простановки ордера на первой бирже
                if token not in leverage_change_history:
                    set_leverage(demo=demo, exc=long_exc_full, symbol=token, leverage=leverage)
                    set_leverage(demo=demo, exc=short_exc_full, symbol=token, leverage=leverage)
                    logger.info(f'Leverage changing...')
    
                    leverage_change_history.append(token)
                    continue
                                                    
                
                if arbitrage_tokens[token]['long_exc'] == 'bybit':
                    first_order_exc = long_exc
                    side = 'buy'
                    price = long_price
                elif arbitrage_tokens[token]['short_exc'] == 'bybit':
                    first_order_exc = short_exc
                    side = 'sell'
                    price = short_price
                else:
                    first_order_exc = long_exc
                    side = 'buy'
                    price = long_price


                price_tick = get_price_scale(coin_information, token, first_order_exc + '_linear')
                if limit_type == 'instant':
                    l_price = Decimal(long_price) + n_ticks * price_tick
                    s_price = Decimal(short_price) - n_ticks * price_tick
                elif limit_type == 'limit':
                    l_price = Decimal(long_price) - n_ticks * price_tick
                    s_price = Decimal(short_price) + n_ticks * price_tick
                else:
                    raise NotImplementedError('Укажите корректный тип лимитного ордера!')

                l_price = l_price.normalize()
                s_price = s_price.normalize()

                price = l_price if first_order_exc == long_exc else s_price
                
                limit_diff = (s_price / l_price - 1) * 100

                
                logger.info(f'[PLACE OPEN ORDER] {volume} {token}; {side} on {first_order_exc}; {usdt_amount=:.2f}')
                logger.info(f'Current: long {long_price}, short {short_price}. Limit: long {l_price}, short {s_price}; diff: {diff:.3f}; limit diff: {limit_diff:.3f}; ')
        
                ts = int(datetime.timestamp(datetime.now()))
                
                resp = place_limit_order(demo=demo, 
                                exc=first_order_exc + '_linear', 
                                symbol=token, 
                                side=side, 
                                volume=volume, 
                                price=price,
                                coin_information=coin_information)
                
                if first_order_exc == 'gate':
                    ord_id = resp['id']
                else:
                    ord_id = resp
                
                redis_order_manager.add_order(exchange=first_order_exc, 
                                              token=token, 
                                              qty=float(volume), 
                                              side=side, 
                                              ord_id=ord_id, 
                                              ts=ts, 
                                              status='placed')
        
        sleep(0.05)

    except (KeyboardInterrupt, CancelledError):
        print('Завершение работы.')
        break

In [ ]:
handle_position(demo, exc='bybit_linear',
                symbol='ADA_USDT',
                order_type='limit',
                coin_information=coin_information)

In [ ]:
token

In [ ]:
pending_orders = redis_order_manager.get_pending_orders()
current_orders = postgre_manager.get_table('current_orders')
current_orders

In [ ]:
pending_orders

In [ ]:
redis_order_manager.add_order(exchange=first_order_exc, 
                                              token=token, 
                                              qty=float(volume), 
                                              side=side, 
                                              ord_id=ord_id, 
                                              ts=ts, 
                                              status='placed')

In [ ]:
redis_order_manager.delete_order(exchange='okx', token='ADA_USDT')

In [ ]:
redis_order_manager.clear_orders_table()

In [ ]:
# План
# 3. Добавить выскакивающее предупреждение, если по какому-то токену идёт большое отклонение на разных биржах, пригодное для арбитража.
# 4. Добавить автоматический стоп-лосс для шортовой сделки и take-profit для лонга
# 7. Можно попробовать реализовать выход из сделок по фандингу, ставя лимитный ордер на 1 - 1.5% от текущей цены - 
# при выплате фандинга цена резко просаживается, а где-то через минуту возвращается обратно. Можно попробовать это использовать.

# Надо посмотреть, считаются ли fees при учёте usdt_amount. Кажется, что у ByBit комиссия уже включена в usdt_amount,
# а у OKX - нет. _order_handler() и _position_handler() OKX usdt_amount считается как qty * price, так что fees не учитываются.

# To do:
# 3. Настроить нормальное логгирование. Продумать те статы, которые мне нужны на экране для анализа.
# 4. Настроить автоматическое обновление статов каждые 10 - 30 минут
# 5. Написать функцию для проверки открытых позиций на бирже и синхронизации с таблицей current_orders в БД.
# 6. Написать функцию, которая будет искать токены с максимальным отклонением от mean значения в текущий момент.

#### Manual open

In [ ]:
redis_order_manager = RedisManager(db_name = 'orders')

In [ ]:
demo = True
token = 'STRK_USDT'
side = 'buy'
usdt_amount = 20
leverage = 2
n_ticks = 2
stop_loss = 0.125

exc = 'bybit'
exc_full = f'{exc}_linear'

dp = coin_information[exc_full][token]['min_qty']

In [ ]:
current_data = redis_price_manager.get_orderbooks(n_levels=1).filter(pl.col('symbol') == token)
if len(current_data) == 0:
    raise Exception('No data!')

set_leverage(demo=demo, exc=exc_full, symbol=token, leverage=leverage)
logger.info(f'Leverage changing...')

In [ ]:
# Открываем лонг-ордер, тип заявки - лимитка на n_ticks лучше текущей цены.
current_data = redis_price_manager.get_orderbooks(n_levels=1).filter(pl.col('symbol') == token)
price = current_data.select(pl.when(side == 'buy').then(pl.col("askprice_0")).otherwise(pl.col("bidprice_0"))).item()
volume = round_size(leverage * usdt_amount / price, dp)

if not is_data_up_to_date(current_data, time_thresh=10):
    logger.error(f'Устаревшие данные по ценам.')
    raise Exception(f'Устаревшие данные по ценам.')
if len(current_data) == 0:
    logger.error(f'No data!')
    raise Exception(f'No data!')

dp = int(coin_information[exc_full][token]['price_scale'])
price_tick = get_price_scale(coin_information, token, exc_full)

# Выбираем цену для установки лимитного ордера на n_ticks тиков лучше текущей цены
if side == 'buy':
    curr_price = current_data.filter((pl.col('exchange') == exc)).select('askprice_0').item()
    price = curr_price - n_ticks * price_tick
    stop_loss = round(price / 2, dp)
elif side == 'sell':
    curr_price = current_data.filter((pl.col('exchange') == exc)).select('bidprice_0').item()
    price = curr_price + n_ticks * price_tick
    stop_loss = round(price * Decimal('1.8'), dp)

ord_id = place_market_order(demo=demo,
                            exc=exc_full,
                            symbol=token,
                            side=side,
                            volume=volume,
                            coin_information=coin_information,
                            stop_loss=stop_loss)
if ord_id:
    if exc == 'gate':
        ord_id = ord_id['id']
    ts = int(datetime.timestamp(datetime.now()))
    redis_order_manager.add_order(exchange=exc, 
                                  token=token, 
                                  qty=volume, 
                                  side=side,
                                  ord_id=ord_id,
                                  ts=ts,
                                  status='placed')

In [ ]:
if exc == 'gate':
    ord_id = ord_id['id']

cancel_id = cancel_order(demo=demo, exc=exc_full, symbol=token, order_id=ord_id)
if cancel_id:
    redis_order_manager.delete_order(exchange=exc, token=token)

In [ ]:
handle_position(demo=demo, exc=exc_full, symbol=token, order_type='market', coin_information=coin_information)

#### Вход в арбитражную сделку:

In [ ]:
"""
logger.info('Запускаем основной цикл программы...')
exc_list = ['bybit', 'okx', 'gate']
mt_list = ['linear', 'spot']

exit_flag = False
signal.signal(signal.SIGINT, signal_handler)
# ====================================
# Параметры запуска
max_mean = 1.2 # Максимальное значение mean в процентах, выше которого все крипто-пары отсекаются
min_std = 0.12 # Показатель std из stats_df для конкретной монеты
std_coef = 2.8 # На сколько умножаем std, вычисляя макс. расстояния от среднего значения
min_edge = 0.0
min_dist = 0.4

demo = False
leverage = 1
max_usdt_order = 20
min_usdt_order = 5

max_orders = 3

bybit_trade = BybitTrade(demo=demo)
okx_trade = OkxTrade(demo=demo)
gate_client = GateTrade(demo=demo)
trading_clients = {'bybit': bybit_trade, 'okx': okx_trade, 'gate': gate_client}

# ====================================
# Основной цикл
while not exit_flag:
    token, data = None, None
    df_in = pd.DataFrame()
    token_dict = dict()
    syms = []

    current_orders = postgre_manager.get_table('current_orders')
    n_current_orders = len(current_orders) // 2
    
    try:
        # Скачиваем текущие данные по ордербукам из Redis
        try:
            current_data = redis_manager.get_orderbooks(n_levels=5)
        except pl.exceptions.ColumnNotFoundError:
            sleep(1)
            continue

        # Проверка на то, что данные являются актуальными, что ни одна из бирж не подвисла
        sorted_df = current_data.sort(by='ts')
        oldest_ts_exc = sorted_df.select("exchange").head(1).item()
        last_entry = sorted_df.filter(pl.col('exchange') == oldest_ts_exc).select("ts").tail(1).item()
        time_delta = int(datetime.timestamp(datetime.now())) - last_entry
        
        # Если данные на бирже последний раз обновлялись больше 20 секунд назад, прерываем работу программы
        if time_delta > 10:
            logger.error(f'Отсутствует соединение с биржей {oldest_ts_exc}.')
            sleep(5)
            continue

        # Здесь будет обработчик текущих лимитных ордеров.
        # Args: order_price, order_side, order_time
        # Если цена, полученная через Redis, доходит до цены ордера - отправляем запрос на подтверждение ордера.
        # Если всё норм и ордер сматчился - отправляем лимитку в обратную сторону.
        # Если прошло 10-15-20 секунд и ордер не выкупили (а цена уехала), отменяем ордер
        # После всех манипуляций возвращаем переменные и order_status в исходное состояние.
        
        # Продолжаем, если данные обновляются регулярно
        stats_data = postgre_manager.get_table('stats_data')
        stats_data = pl.from_pandas(stats_data)
        df_in = find_tokens_to_open_order(current_data=current_data, 
                                          stats_data=stats_data, 
                                          max_mean=max_mean, 
                                          min_std=min_std, 
                                          std_coef=std_coef, 
                                          min_edge=min_edge,
                                          min_dist=min_dist)
        
        # Создадим словарь с токенами для того, чтобы посчитать, сколько usdt ещё доступно для открытия ордера.
        if not df_in.is_empty():
            # Если в df_in есть несколько вариантов для одного токена:
            n_counts = df_in["token"].value_counts(sort=True).select('count').head(1).item()
            if n_counts > 1:
                print('Есть несколько вариантов для одного токена')
                df_in_copy = df_in.clone()
                if bybit_diff - 0.1 < other_diff:
                    df_in = df_in.filter((pl.col("token") != token) | ((pl.col("long_exc") != "bybit") & (pl.col("short_exc") != "bybit")))
                else:
                    df_in = df_in.filter((pl.col("token") != token) | ((pl.col("long_exc") == "bybit") | (pl.col("short_exc") == "bybit")))
                
                exit_flag = True
                break
            
            token_list = df_in['token'].to_list()
                        
            for token in token_list:
                if token in current_orders['token'].to_list():
                    token_dict[token] = max_usdt_order - current_orders[current_orders['token'] == token].iloc[0]['usdt_amount']
                else:
                    token_dict[token] = max_usdt_order
        else:
            token_list = []

        for row in df_in.to_dicts():            
            token = row["token"]
            long_exc = row["long_exc"]
            short_exc = row["short_exc"]
                        
            if token_dict[token] < min_usdt_order:
                continue
            
            usdt_remain = 0 # Сбрасываем настройки для безопасности
            
            min_edge = row["thresh"]
            price_long = row['ask_long']
            price_short = row['bid_short']
            thresh = row['thresh']
        
            df_long = current_data.filter((pl.col("exchange") == long_exc) & (pl.col("symbol") == token))
            df_short = current_data.filter((pl.col("exchange") == short_exc) & (pl.col("symbol") == token))
            ask_price = df_long.select('askprice_0', 'askprice_1', 'askprice_2', 'askprice_3', 'askprice_4').row(0)
            ask_size = df_long.select('askvolume_0', 'askvolume_1', 'askvolume_2', 'askvolume_3', 'askvolume_4').row(0)
            bid_price = df_short.select('bidprice_0', 'bidprice_1', 'bidprice_2', 'bidprice_3', 'bidprice_4').row(0)
            bid_size = df_short.select('bidvolume_0', 'bidvolume_1', 'bidvolume_2', 'bidvolume_3', 'bidvolume_4').row(0)
            long_ob = [[price, size] for price, size in zip(ask_price, ask_size)]
            short_ob = [[price, size] for price, size in zip(bid_price, bid_size)]
            
            usdt_remain = token_dict[token]
            edge = max(thresh, min_edge)
            
            res = get_open_volume(long_orderbook=long_ob, 
                                  short_orderbook=short_ob, 
                                  min_edge=edge, 
                                  max_usdt=usdt_remain, # Умножаем на 2, чтобы был запас по ликвидности
                                  debug=False)
            
            # Если в ордербуке достаточно ликвидности для открытия ордера с заданным diff, открываем сделку
            # Проверяем, что у нас есть свободные слоты на открытие ордеров, или что ордер уже открыт и мы можем докупиться
            if res['usdt_amount'] >= min_usdt_order and (n_current_orders < max_orders or token in current_orders['token'].to_list()):
                # Если мне предлагают открыть ордер в направлении, отличном от уже существующего на бирже
                if token in current_orders['token'].to_list():
                    try:
                        long_position_side = current_orders[(current_orders['token'] == token) & 
                                                            (current_orders['exchange'] == long_exc)]['order_side'].item()
                        short_position_side = current_orders[(current_orders['token'] == token) & 
                                                            (current_orders['exchange'] == short_exc)]['order_side'].item()
    
                        if long_position_side == 'sell' or short_position_side == 'buy':
                            continue
                    except ValueError:
                        # Это означает, что мне предлагают открыть ещё одну сделку на другой бирже
                        continue
                
                
                long_exc_full = long_exc + '_linear'
                short_exc_full = short_exc + '_linear'
                
                ct = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                min_qty_step = get_step_info(coin_information, token, long_exc_full, short_exc_full)
                curr_diff = res['edge']
                volume = round_volume(volume=res['volume'], qty_step=min_qty_step) # Делим на 2, избавляясь от коэффа
                
                # Если при округлении цена упала меньше минимально возможной
                if volume < min_qty_step:
                    continue
                
                usdt_amount = volume / Decimal(res['volume']) * Decimal(res['usdt_amount'])
                mean = row['mean']
                std = row['std']
                deviation = (curr_diff - mean) / std

                long_price = long_ob[0][0]
                short_price = short_ob[0][0]
                
                # Сначала на обеих биржах меняем плечо, чтобы если есть проблемы со связью,
                #   всё отвалилось на этапе изменения плеча, а не после простановки ордера на первой бирже
                if token not in leverage_change_history:
                    set_leverage(demo=demo, exc=long_exc_full, symbol=token, leverage=leverage)
                    set_leverage(demo=demo, exc=short_exc_full, symbol=token, leverage=leverage)
                    logger.info(f'Leverage changing...')

                    leverage_change_history.append(token)
                    continue
                
                # Для дебага добавляем ордербук, на основании которого был открыт ордер, в список, хранящий историю
                orders_history.append({'time': ct, 'row': row, 'res': res, 'qty_step': min_qty_step})
                with open("./data/open_order_history.pkl", "wb") as f:
                    pickle.dump(orders_history, f)

                logger.info(f'[PLACE OPEN ORDER] {volume} {token}; diff: {curr_diff:.3f}, {thresh=:.3f}, {mean=:.3f}, {std=:.3f}; {deviation=:.3f}')
                logger.info(f'Цены во время простановки ордера. long: {price_long} ({long_exc}), short: {price_short} ({short_exc}); {usdt_amount=:.2f}')
                                    
                # Открываем лонг-ордер
                long_order_resp = place_market_order(demo=demo, exc=long_exc_full, symbol=token, side='buy', volume=volume)
                logger.info(f'Лонг ордер ({long_exc}) отправлен на биржу.')
                
                # Открываем шорт-ордер
                short_order_resp = place_market_order(demo=demo, exc=short_exc_full, symbol=token, side='sell', volume=volume)
                logger.info(f'Шорт ордер ({short_exc}) отправлен на биржу.')
                

                # Обрабатываем полученные ответы и заносим ордеры в БД
                handle_position(demo=demo, exc=long_exc_full, symbol=token, order_type='market')
                handle_position(demo=demo, exc=short_exc_full, symbol=token, order_type='market')
        
        sleep(0.02)
    except (KeyboardInterrupt, CancelledError):
        print('Завершение работы.')
        break
    # except RuntimeError as e:
    #     print(f"Ошибка выполнения: {e}")


"""